# Montreal historical weather data from weather.gc.ca

This code automates the downolad of all csv files for weather data for Montreal from [weather.gc.ca](http://climate.weather.gc.ca/).
Files describing hourly weather information are available in monthly files, combined in a single file by the end of this notebook. Selenium is used to automate the control of the browser.

In [1]:
import time
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import *
from selenium import webdriver  # Selinium requires also a driver to be able to control a browser, 
                                # to be installed in the project directory from
                                #  https://selenium-python.readthedocs.io/installation.html

In [2]:
# Dictionary mapping months' numbers from pervious function with their names
month_dictio ={ 1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June',
    7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}
inv_month_dictio = {v: k for k, v in month_dictio.items()}

def generate_dates_df(start_date,end_date,date_format='%Y-%m-%d'):
    """Generate a dataframe of the monthly spaced dates between start_date and end_date """
    """(since the available data is monthly stored on the targeted website) """
    
    s_date = datetime.datetime.strptime(start_date,date_format )
    e_date = datetime.datetime.strptime(end_date, date_format)
    counter = s_date
    dates = []
    while counter <= e_date:
        dates.append(counter.date())
        counter += relativedelta(months=1) # advance one month each iteration
    dates_list = [str(date) for date in dates]
    # creates a pandas dataframe of dates
    dates_df = pd.DataFrame(dates_list,columns=['date'])
    ff = pd.to_datetime(dates_df.date, format=date_format)
    for att in ('Day','Month','Year'):
        dates_df[att] = np.zeros(len(dates_list),int)
        for ind in ff.index:
            dates_df.loc[ind,att] = getattr(ff[ind],att.lower())
    dates_df['Month'] = dates_df['Month'].map(month_dictio)
    return dates_df


In [40]:
driver = webdriver.Chrome('chromedriver_win32/chromedriver')
driver.get('http://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2013-02-13%7C2018-09-18&dlyRange=2013-02-14%7C2018-09-18&mlyRange=%7C&StationID=51157&Prov=QC&urlExtension=_e.html&searchType=stnName&optLimit=specDate&StartYear=2010&EndYear=2018&selRowPerPage=25&Line=0&searchMethod=contains&Month=4&Day=1&txtStationName=montreal&timeframe=1&Year=2014')
start_date = '2014-04-01
end_date = '2018-08-01'
dates_list_df = generate_dates_df(start_date,end_date,'%Y-%m-%d')
for ind, row in dates_list_df.iterrows():
    D = row['Day']
    M = row['Month']
    Y = row['Year']
    year = driver.find_element_by_id('Year1')
    year.send_keys(Y)
    month = driver.find_element_by_id('Month1')
    month.send_keys(M)
    day = driver.find_element_by_id('Day1')
    day.send_keys(D)
    day.submit()
    download_btn = driver.find_element_by_xpath('//*[@id="climateNav"]/div[3]/section/div[1]/form/input[6]')
    download_btn.click()

This downloaded all 53 csv files from start_date to end_date in less than 2 min

In [2]:
from os import listdir
from os.path import isfile, join

Load all files, merge and sort them

In [3]:
all_csvs = [f for f in listdir('../Weather_Data/raw') if isfile(join('../Bixi/Weather_Data/raw', f))]

In [5]:
all_weather_data = pd.read_csv('../Weather_Data/raw/'+all_csvs[0], skiprows=15)
for f in all_csvs:
    aux_df = pd.read_csv('../Weather_Data/raw/'+f, skiprows=15)
    all_weather_data = all_weather_data.append(aux_df,ignore_index=True)

In [6]:
all_weather_data.tail()

,Date/Time,Year,Month,Day,Time,Temp (°C),Temp Flag,Dew Point Temp (°C),Dew Point Temp Flag,Rel Hum (%),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
39475,2017-12-31 19:00,2017,12,31,19:00,-21.9,NaN,-27.4,NaN,62.0,...,NaN,24.1,NaN,102.33,NaN,NaN,NaN,-33.0,NaN,Clear
39476,2017-12-31 20:00,2017,12,31,20:00,-22.8,NaN,-28.0,NaN,63.0,...,NaN,24.1,NaN,102.35,NaN,NaN,NaN,-33.0,NaN,NaN
39477,2017-12-31 21:00,2017,12,31,21:00,-22.9,NaN,-28.1,NaN,64.0,...,NaN,24.1,NaN,102.36,NaN,NaN,NaN,-32.0,NaN,NaN
39478,2017-12-31 22:00,2017,12,31,22:00,-23.4,NaN,-28.3,NaN,65.0,...,NaN,24.1,NaN,102.36,NaN,NaN,NaN,-34.0,NaN,Clear
39479,2017-12-31 23:00,2017,12,31,23:00,-23.2,NaN,-28.2,NaN,64.0,...,NaN,24.1,NaN,102.37,NaN,NaN,NaN,-33.0,NaN,NaN


In [7]:
all_weather_data['Date/Time'] = all_weather_data['Date/Time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M'))
all_weather_data = all_weather_data.sort_values(by=['Date/Time'])
all_weather_data.tail()

,Date/Time,Year,Month,Day,Time,Temp (°C),Temp Flag,Dew Point Temp (°C),Dew Point Temp Flag,Rel Hum (%),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
27763,2018-08-31 19:00:00,2018,8,31,19:00,22.6,NaN,13.2,NaN,55.0,...,NaN,24.1,NaN,101.88,NaN,26.0,NaN,NaN,NaN,Mostly Cloudy
27764,2018-08-31 20:00:00,2018,8,31,20:00,21.8,NaN,12.6,NaN,55.0,...,NaN,24.1,NaN,101.89,NaN,NaN,NaN,NaN,NaN,NaN
27765,2018-08-31 21:00:00,2018,8,31,21:00,21.6,NaN,12.2,NaN,55.0,...,NaN,24.1,NaN,101.90,NaN,NaN,NaN,NaN,NaN,NaN
27766,2018-08-31 22:00:00,2018,8,31,22:00,21.8,NaN,13.9,NaN,60.0,...,NaN,24.1,NaN,101.89,NaN,25.0,NaN,NaN,NaN,Mostly Cloudy
27767,2018-08-31 23:00:00,2018,8,31,23:00,21.1,NaN,16.4,NaN,74.0,...,NaN,24.1,NaN,101.93,NaN,26.0,NaN,NaN,NaN,NaN


In [8]:
# Change Time column format and name to be consistent with the trips data
all_weather_data['Time'] = all_weather_data['Date/Time'].apply(lambda x: x.hour)
all_weather_data = all_weather_data.rename(columns = {'Time':'Hour'})
# drop all-nan-columns
all_weather_data = all_weather_data.drop(columns=['Temp Flag', 'Dew Point Temp Flag','Rel Hum Flag','Wind Dir Flag', 'Wind Spd Flag', 'Wind Chill', 'Stn Press Flag', 'Visibility Flag','Hmdx Flag','Wind Chill Flag'])

In [105]:
#export all to new csv
all_weather_data.to_csv('../Weather_Data/raw/yearly/all_weather_data_montreal.csv',index=False)


In [14]:
#export each year seperatly to new csv
for y in range(2014,2019):
    all_weather_data[all_weather_data.Year==y].to_csv('../Weather_Data/raw/yearly/weather' + str(y) + '.csv',index=False)